In [94]:
from absbox import API, mkDeal,EnginePath
import pandas as pd 
#localAPI = API("http://localhost:8081/", lang="english",check=False)
name = "Deal"
dates = {
    "cutoff":"2024-10-31",
    "closing":"2024-10-31",
    "firstPay":"2024-11-30",
    "stated": "2027-10-31",
    "poolFreq": "MonthEnd",
    "payFreq": "MonthEnd"
}

amortising_loan_monthly = ["Mortgage"
            ,{"originBalance": 6922.22
            ,"originRate": ["fix",0.16]
            ,"originTerm": 12
            ,"freq": "Monthly"
            ,"type": "Even"
            ,"originDate": "2024-10-31"}
            ,{"currentBalance": 6922.22
            ,"currentRate": 0.16
            ,"remainTerm": 12
            ,"status": "current"}]
amortising_loan_quarterly = ["Mortgage"
            ,{"originBalance":  2_966.67    
            ,"originRate": ["fix",0.16]
            ,"originTerm": 4
            ,"freq": "Quarterly"
            ,"type": "Even"
            ,"originDate": "2024-10-31"}
            ,{"currentBalance": 2_966.67
            ,"currentRate": 0.16
            ,"remainTerm": 4
            ,"status": "current"}]

bullet_loans = ["Loan"
            ,{"originBalance": 12_222.22
            ,"originRate": ["fix",0.16]
            ,"originTerm": 6
            ,"freq": "Monthly"
            ,"type": "i_p"
            ,"originDate": "2024-10-31"}
            ,{"currentBalance": 12_222.22
            ,"currentRate": 0.16
            ,"remainTerm": 6
            ,"status": "current"}]

j = {"pool_amortizing": {'assets':[amortising_loan_monthly]*100 + [amortising_loan_quarterly]*100},"pool_bullet": {'assets':[bullet_loans]*10}}


accounts = {
    "collections_interest": {"balance": 0},
    "collections_principal_amortizing": {"balance": 0},
    "collections_principal_bullet": {"balance": 0},
    "revolBuyAcc1": {"balance": 0},
    "revolBuyAcc2": {"balance": 0},
    "dealAcc": {"balance": 0},
}
bonds = {
    "A1": {
        "balance": 1_000_000.00,
        "rate": 0.114,
        "originBalance": 30_000_000,
        "originRate": 0.114,
        "startDate": "2024-10-31",
        "rateType": {"floater":[0.049, "SOFR1M",0.065,"MonthEnd"]},
        "maturityDate": "2027-10-31",
        "bondType": {"Sequential":None},
    },
    "EQ": {
        "balance":  111_111.00,
        "rate": 0.0,
        "originBalance": 111_111.00,
        "originRate": 0.0,
        "startDate": "2024-10-31",
        "maturityDate": "2027-10-31",
        "rateType": {"Fixed": 0.00},
        "bondType": {"Equity": None},
    },
}

                                

fees = {"utilisation_fee": {"type": {"annualPctFee": [("excess",("originalBondBalance", "A1"),("bondBalance", "A1"),),0.007],},
                            "feeStart": "2025-01-31","feeEnd": "2026-10-31"},
        "technology_fee": {"type": {"annualPctFee": [("poolBalance",),0.005],},
                            "feeStart": "2025-01-31"},}
waterfall = {
                    "closingDay":[
                        ["fundWith", "revolBuyAcc1", "A1",{"formula": ("const",  2_000_000.00)}], 
                        #["fundWith", "revolBuyAcc2", "A1",{"formula": ("const", 220_000.00)}],
                        ["fundWith", "revolBuyAcc1", "EQ",{"formula": ("const",  197_777.78)}],
                        ["fundWith", "revolBuyAcc2", "EQ",{"formula": ("const",  24_444.44)}],
                        ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc1",None,"Pool1","pool_amortizing"],
                        ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc2",None,"Pool2","pool_bullet"],

                    ],
                    "Accelerated":[
                                ["calcInt", "A1"],
                                ["fundWith", "revolBuyAcc1", "A1",{"formula": ("const",  2_670_000.00+330_000)}], 
                                #["fundWith", "revolBuyAcc2", "A1",{"formula": ("const", 330_000.00)}],
                                ["fundWith", "revolBuyAcc1", "EQ",{"formula": ("const",  296_666.67)}],
                                ["fundWith", "revolBuyAcc2", "EQ",{"formula": ("const",  36_666.67)}],
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee","technology_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ['transfer','collections_principal_amortizing','revolBuyAcc1'],
                                ['transfer','collections_principal_bullet','revolBuyAcc2'],
                                ["transfer","collections_interest","revolBuyAcc1",{
                                "formula": ("-",("bondBalance","A1"),
                                            ("factor",
                                             ("sum",("poolBalance","pool_amortizing"),
                                                    ("poolBalance","pool_bullet"),
                                              ("accountBalance","revolBuyAcc1","revolBuyAcc2","collections_principal_amortizing","collections_principal_bullet"),),
                                             0.89,),),
                                    "support": ["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]},
                                    ],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc1",None,"Pool1","pool_amortizing"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc2",None,"Pool2","pool_bullet"],
                                ["payIntResidual", "collections_interest", "EQ"],
                                ],
                    "revolving":[
                                ["calcInt", "A1"],
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee","technology_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ['transfer','collections_principal_amortizing','revolBuyAcc1'],
                                ['transfer','collections_principal_bullet','revolBuyAcc2'],
                                ["transfer","collections_interest","revolBuyAcc1",{
                                "formula": ("-",("bondBalance","A1"),
                                            ("factor",
                                             ("sum",("poolBalance","pool_amortizing"),
                                                    ("poolBalance","pool_bullet"),
                                              ("accountBalance","revolBuyAcc1","revolBuyAcc2","collections_principal_amortizing","collections_principal_bullet"),),
                                             0.9,),)
                                    ,"support": ["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]},
                                    ],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc1",None,"Pool1","pool_amortizing"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc2",None,"Pool2","pool_bullet"],
                                ["payIntResidual", "collections_interest", "EQ"],
                                ],
                    "amortizing":[
                                ["calcInt", "A1"],  # actions if deal is in Amortizing status
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee","technology_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ['transfer','collections_principal_amortizing','dealAcc'],
                                ['transfer','collections_principal_bullet','dealAcc'],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                                ["payIntResidual", "collections_interest", "EQ"],
                                ],

                    "cleanUp": [
                            ['transfer','collections_principal_amortizing','dealAcc'],
                            ['transfer','collections_principal_bullet','dealAcc'],
                            ["accrueAndPayInt", "collections_interest", ["A1"]],
                            ['transfer','collections_interest','dealAcc'],
                            ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                            ["payPrin", "dealAcc", ["EQ"]],
                            ["payIntResidual", "dealAcc", "EQ"],
                            ],

                    "Defaulted": [
                            ['transfer','collections_principal_amortizing','dealAcc'],
                            ['transfer','collections_principal_bullet','dealAcc'],
                            ["accrueAndPayInt", "collections_interest", ["A1"]],
                            ['transfer','collections_interest','dealAcc'],
                            ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                            ["payIntResidual", "dealAcc", "EQ"],
                            ["payPrin", "dealAcc", ["EQ"]],],
                            
                }

collects = [
            [["pool_amortizing"],"CollectedPrincipal", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedRecoveries", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedPrepayment", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedInterest", "collections_interest"],
            [["pool_bullet"],"CollectedPrincipal", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedRecoveries", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedPrepayment", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedInterest", "collections_interest"],
        ]
defrate3periods = \
   [ ("ratio",("curPoolCollectionStats", None, _*-1, "Defaults"),("curPoolCollectionStats", None, _*-1, "BegBalance"))
   for _ in range(3) ]

trigger = {
            "AfterCollect": {
                "revolving_trigger": {
                    "condition": ["any", [">", "2026-01-31"]],
                    "effects": ("newStatus", "Revolving"),
                    "status": False,
                    "curable": False,
                },
                "amortization_trigger": {
                    "condition": ["any", [">=", "2026-10-31"]],
                    "effects": ("newStatus", "Amortizing"),
                    "status": False,
                    "curable": False,
                },
                "DefaultRate_trigger": { #Rolling Default rate trigger
            "condition": ["all", 
                          [">=", "2025-01-31"],
                          [("avgRatio",*defrate3periods),">", 0.06],
            ],
            "effects": ("newStatus", "Defaulted"),
            "status": False,
            "curable": False,
        },},
        "AfterDistribution": {
                "LTV_trigger": {
                    "condition": ["all",
                                  [">", "2024-10-31"],
                                  [
                        (
                            "ratio",
                            ("bondBalance","A1"),
                            ("sum",
                             ("poolBalance","pool_amortizing"),
                             ("poolBalance","pool_bullet"),
                             ("accountBalance","revolBuyAcc1","revolBuyAcc2","dealAcc","collections_principal_amortizing","collections_principal_bullet"),
                             ),
                        ),
                            ">",
                        0.9,
                    ],],
                    "effects": ("newStatus", "Defaulted"),
                    "status": False,
                    "curable": True,
                },
            }, 
        }

deal_data = {
            "name": name,
            "dates": dates,
            "pool": j,
            "accounts": accounts,
            "fees": fees,
            "bonds": bonds,
            "waterfall": waterfall,
            "collect": collects,
            "trigger": trigger,
            "status": ("PreClosing","Accelerated"),
        }

deal = mkDeal(deal_data)
revol_asset1 = [amortising_loan_monthly] +[amortising_loan_quarterly]
#revol_asset1 = [amortising_loan_quarterly]
revol_asset2 = bullet_loans

cdr_assump = 0

cdr_assump_1 = 0

revolvingPool1 = (["constant",*revol_asset1]
                 ,("Pool",("Mortgage",{"CDR":cdr_assump_1},None,None,None
                                                                   )
                   ,None
                   ,None))

revolvingPool2 = (["constant",revol_asset2]
                 ,("Pool",("Loan",{"CDR":cdr_assump_1},None,None,None)
                   ,None
                   ,None))

amortisingPoolAssump = ("Pool",("Mortgage",{"CDR":cdr_assump},None,None,None)
                   ,None
                   ,None)

bulletPoolAssump = ("Pool",("Loan",{"CDR":cdr_assump},None,None,None)
                   ,None
                   ,None)

localAPI = API(EnginePath.DEV, lang="english",check=False)
r = localAPI.run(deal,
                 poolAssump = ("ByPoolId",
                                    {"pool_amortizing":amortisingPoolAssump
                                    ,"pool_bullet":bulletPoolAssump}
                                    ),
         runAssump=[#tuple(trigger_inspect_creation(deal)),
             ("interest", ("SOFR1M", 0.049), ("EURIBOR1M", 0.0858)),
             #("inspect",("MonthEnd",("bondBalance", "A1")),
              #("MonthEnd",("poolBalance","pool_amortizing")),
              #("MonthEnd",("poolBalance","pool_bullet")),
              #("MonthEnd", ("sum",
              #               ("poolBalance","pool_amortizing"),
              #               ("poolBalance","pool_bullet"),
              #               ("accountBalance","revolBuyAcc1","revolBuyAcc2","dealAcc","collections_principal_amortizing","collections_principal_bullet"),
              #               ))
              #("MonthEnd",("curPoolCollectionStats", None, 0, "Principal")),
              #("MonthEnd",("curPoolCollectionStats", None, 0, "Interest")),
              #("MonthEnd",("curPoolCollectionStats", None, 0, "Defaults"))
              #,("MonthEnd",("accountBalance","revolBuyAcc1","revolBuyAcc2",))
             #,("MonthEnd",("curPoolCollectionStats", None, 0, "BegBalance"))
              #                ),
              ("call", {"afterDate":"2027-10-31"}),
              ("revolving",{"Pool1":revolvingPool1,"Pool2":revolvingPool2})
              #,("stop","2024-12-01")                                
             
         ]
         ,read=True)

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.28.7, server:0.29.19

Warning Message from server:
Bond EQ is not paid off
Account dealAcc has cash to be distributed

In [95]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,intDue,intOverInt,factor,memo
date,,,,,,,,,
2024-10-31,3000000.00,0.00,-2000000.00,0.000,0.00,0,0,0.100000,"<FundWith:A1,2000000.00>"
2024-11-30,6000000.00,28109.58,-3000000.00,0.114,28109.58,0,0,0.200000,"[<FundWith:A1,3000000.00>, <PayInt:A1>]"
2024-12-31,9000000.00,58093.15,-3000000.00,0.114,58093.15,0,0,0.300000,"[<FundWith:A1,3000000.00>, <PayInt:A1>]"
2025-01-31,12000000.00,87139.72,-3000000.00,0.114,87139.72,0,0,0.400000,"[<FundWith:A1,3000000.00>, <PayInt:A1>]"
2025-02-28,15000000.00,104942.46,-3000000.00,0.114,104942.46,0,0,0.500000,"[<FundWith:A1,3000000.00>, <PayInt:A1>]"
2025-03-31,18000000.00,145232.87,-3000000.00,0.114,145232.87,0,0,0.600000,"[<FundWith:A1,3000000.00>, <PayInt:A1>]"
2025-04-30,21000000.00,168657.53,-3000000.00,0.114,168657.53,0,0,0.700000,"[<FundWith:A1,3000000.00>, <PayInt:A1>]"
2025-05-31,24000000.00,203326.02,-3000000.00,0.114,203326.02,0,0,0.800000,"[<FundWith:A1,3000000.00>, <PayInt:A1>]"
2025-06-30,27000000.00,224876.71,-3000000.00,0.114,224876.71,0,0,0.900000,"[<FundWith:A1,3000000.00>, <PayInt:A1>]"


In [97]:
r['pool']['flow']['pool_amortizing']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-10-31,988889.00,0.00,0.00,0,0,0,0,0.160000,None,None,0.000000e+00,0,0,0,0,0
2024-11-30,3000778.13,185888.64,29741.07,0,0,0,0,0.160000,None,None,1.858886e+05,0,0,0,0,0
2024-12-31,6097648.39,389245.29,59798.98,0,0,0,0,0.160000,None,None,5.751339e+05,0,0,0,0,0
2025-01-31,8944241.22,843500.11,127288.63,0,0,0,0,0.160000,None,None,1.418634e+06,0,0,0,0,0
2025-02-28,12017560.82,1109985.57,161866.18,0,0,0,0,0.160000,None,None,2.528620e+06,0,0,0,0,0
2025-03-31,15084523.21,1384980.46,194550.02,0,0,0,0,0.160000,None,None,3.913600e+06,0,0,0,0,0
2025-04-30,17867134.92,1936238.02,258414.25,0,0,0,0,0.160000,None,None,5.849838e+06,0,0,0,0,0
2025-05-31,20890287.71,2288702.38,295463.05,0,0,0,0,0.160000,None,None,8.138540e+06,0,0,0,0,0
2025-06-30,23898991.36,2654364.21,330778.40,0,0,0,0,0.160000,None,None,1.079290e+07,0,0,0,0,0


In [98]:
r['pool']['flow']['pool_bullet']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,
2024-10-31,122222.20,0.00,0.00,0,0,0,0,0.160000,0.00,0,0,0,0,0
2024-11-30,146666.64,0.00,1928.76,0,0,0,0,0.160000,0.00,0,0,0,0,0
2024-12-31,183333.31,0.00,2475.16,0,0,0,0,0.160000,0.00,0,0,0,0,0
2025-01-31,219999.98,0.00,2989.49,0,0,0,0,0.160000,0.00,0,0,0,0,0
2025-02-28,256666.65,0.00,3166.32,0,0,0,0,0.160000,0.00,0,0,0,0,0
2025-03-31,293333.32,0.00,3921.72,0,0,0,0,0.160000,0.00,0,0,0,0,0
2025-04-30,183333.35,146666.64,4371.85,0,0,0,0,0.160000,146666.64,0,0,0,0,0
2025-05-31,329999.99,36666.67,4870.11,0,0,0,0,0.160000,183333.31,0,0,0,0,0
2025-06-30,366666.66,36666.67,5400.53,0,0,0,0,0.160000,219999.98,0,0,0,0,0


In [99]:
from absbox import unifyTs,readAccsCf

In [100]:
readAccsCf(r['agg_accounts'])

Account    collections_interest                            \
Field             begin balance        change end balance   
date                                                        
2024-10-31                  0.0  0.000000e+00         0.0   
2024-11-30                  0.0 -3.637979e-12         0.0   
2024-12-31                  0.0  5.456968e-12         0.0   
2025-01-31                  0.0  7.275958e-12         0.0   
2025-02-28                  0.0 -7.275958e-12         0.0   
2025-03-31                  0.0 -1.455192e-11         0.0   
2025-04-30                  0.0  1.455192e-11         0.0   
2025-05-31                  0.0 -1.182343e-11         0.0   
2025-06-30                  0.0  7.275958e-11         0.0   
2025-07-31                  0.0 -2.910383e-11         0.0   
2025-08-31                  0.0  2.910383e-11         0.0   
2025-09-30                  0.0 -1.455192e-11         0.0   
2025-10-31                  0.0 -1.455192e-11         0.0   
2025-11-30                  0.0 -5.820766e-11         0.0   
2025-12-31                  0.0 -5.820766e-11         0.0   
2026-01-31                  0.0  2.910383e-11         0.0   
2026-02-28                  0.0  0.000000e+00         0.0   
2026-03-31                  0.0 -8.731149e-11         0.0   
2026-04-30                  0.0  2.910383e-11         0.0   
2026-05-31                  0.0 -5.820766e-11         0.0   
2026-06-30                  0.0  2.910383e-11         0.0   
2026-07-31                  0.0 -2.910383e-11         0.0   
2026-08-31                  0.0  0.000000e+00         0.0   
2026-09-30                  0.0  8.731149e-11         0.0   
2026-10-31                  0.0  2.910383e-11         0.0   
2026-11-30                  0.0  5.820766e-11         0.0   
2026-12-31                  0.0 -2.910383e-11         0.0   
2027-01-31                  0.0  2.910383e-11         0.0   
2027-02-28                  0.0 -2.910383e-11         0.0   
2027-03-31                  0.0  5.820766e-11         0.0   
2027-04-30                  0.0  1.455192e-11         0.0   
2027-05-31                  0.0  0.000000e+00         0.0   
2027-06-30                  0.0 -1.455192e-11         0.0   
2027-07-31                  0.0  0.000000e+00         0.0   
2027-08-31                  0.0  0.000000e+00         0.0   
2027-09-30                  0.0 -1.818989e-12         0.0   

Account    collections_principal_amortizing                     \
Field                         begin balance change end balance   
date                                                             
2024-10-31                              0.0    0.0         0.0   
2024-11-30                              0.0    0.0         0.0   
2024-12-31                              0.0    0.0         0.0   
2025-01-31                              0.0    0.0         0.0   
2025-02-28                              0.0    0.0         0.0   
2025-03-31                              0.0    0.0         0.0   
2025-04-30                              0.0    0.0         0.0   
2025-05-31                              0.0    0.0         0.0   
2025-06-30                              0.0    0.0         0.0   
2025-07-31                              0.0    0.0         0.0   
2025-08-31                              0.0    0.0         0.0   
2025-09-30                              0.0    0.0         0.0   
2025-10-31                              0.0    0.0         0.0   
2025-11-30                              0.0    0.0         0.0   
2025-12-31                              0.0    0.0         0.0   
2026-01-31                              0.0    0.0         0.0   
2026-02-28                              0.0    0.0         0.0   
2026-03-31                              0.0    0.0         0.0   
2026-04-30                              0.0    0.0         0.0   
2026-05-31                              0.0    0.0         0.0   
2026-06-30                              0.0    0.0         0.0   
2026-07-31                          

In [101]:
u = unifyTs(r['result']['inspect'].values())
u.head()

KeyError: 'inspect'

In [90]:
# backout trigger value
u['<CurrentBondBalanceOf:A1>']/u.iloc[:,1:].sum(axis=1)

Date
2024-10-31    0.900000
2024-11-30    0.899520
2024-12-31    0.899304
2025-01-31    0.896579
2025-02-28    0.894837
2025-03-31    0.894038
2025-04-30    0.891884
2025-05-31    0.890317
2025-06-30    0.890152
2025-07-31    0.890124
2025-08-31    0.890111
2025-09-30    0.890102
2025-10-31    0.890094
2025-11-30    0.890087
2025-12-31    0.890081
2026-01-31    0.890076
2026-02-28    0.890076
2026-03-31    0.890076
2026-04-30    0.890076
2026-05-31    0.890076
2026-06-30    0.890076
2026-07-31    0.890076
2026-08-31    0.890076
2026-09-30    0.890076
2026-10-31    0.864686
2026-11-30    0.831392
2026-12-31    0.784640
2027-01-31    0.725514
2027-02-28    0.643357
2027-03-31    0.514299
2027-04-30    0.334004
2027-05-31    0.019523
2027-06-30    0.000000
2027-07-31    0.000000
2027-08-31    0.000000
2027-09-30    0.000000
2027-10-31    0.000000
dtype: float64

In [102]:
r['triggers']['EndDistributionWF']['LTV_trigger']

,status,memo
date,,
2024-11-30,False,<Tag:All:2024-11-30 G 2024-10-31|0.899519 G 0....
2024-12-31,False,<Tag:All:2024-12-31 G 2024-10-31|0.899303 G 0....
2025-01-31,False,<Tag:All:2025-01-31 G 2024-10-31|0.896578 G 0....
2025-02-28,False,<Tag:All:2025-02-28 G 2024-10-31|0.894836 G 0....
2025-03-31,False,<Tag:All:2025-03-31 G 2024-10-31|0.894037 G 0....
2025-04-30,False,<Tag:All:2025-04-30 G 2024-10-31|0.891884 G 0....
2025-05-31,False,<Tag:All:2025-05-31 G 2024-10-31|0.890317 G 0....
2025-06-30,False,<Tag:All:2025-06-30 G 2024-10-31|0.890151 G 0....
2025-07-31,False,<Tag:All:2025-07-31 G 2024-10-31|0.890123 G 0....


In [103]:
r['result']['waterfall']

,Date,Waterfall Location
0,2024-10-31,<OnClosingDay>
1,2024-11-30,<DistributionDay:<DealAccelerated>>
2,2024-12-31,<DistributionDay:<DealAccelerated>>
3,2025-01-31,<DistributionDay:<DealAccelerated>>
4,2025-02-28,<DistributionDay:<DealAccelerated>>
5,2025-03-31,<DistributionDay:<DealAccelerated>>
6,2025-04-30,<DistributionDay:<DealAccelerated>>
7,2025-05-31,<DistributionDay:<DealAccelerated>>
8,2025-06-30,<DistributionDay:<DealAccelerated>>
9,2025-07-31,<DistributionDay:<DealAccelerated>>
